到目前为止，我们讨论了如何处理数据，以及如何构建、训练和测试深度学习模型。然而，有时我们对所学的模型足够满意，我们希望保存训练的模型以备将来在各种环境中使用（甚至可能在部署中进行预测）。此外，当运行一个耗时较长的训练过程时，最佳的做法是定期保存中间结果（检查点），以确保在服务器电源被不小心断掉时不会损失几天的计算结果。因此，现在是时候学习如何加载和存储权重向量和整个模型。本节将讨论这些问题。

# 加载和保存张量
对于单个张量，可以直接调用load和save函数分别读写他们。这两个函数要求我们提供一个名称，save要求将要保存的变量作为输入

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
x =torch.arange(4)
torch.save(x,'x-file')
x

tensor([0, 1, 2, 3])

In [4]:
x2 =torch.load('x-file')
x2

tensor([0, 1, 2, 3])

可以存储一个张量列表，然后把他们读回内存

In [5]:
y =torch.zeros(4)
torch.save([x,y],'x-file')
x2,y2 =torch.load('x-file')
x2,y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以写入或读取从字符串映射到张量的字典。当我们要读取或写入模型中的所有权重时，这很方便。

In [6]:
mydict ={'x':1,'y':2}
torch.save(mydict,'mydict')
mydict2 =torch.load('mydict')
mydict2

{'x': 1, 'y': 2}

# 加载和保存模型数据
保存单个权重向量（或其他张量）确实是有用的，但是如果我们想保存整个模型，并在以后加载它们，单独保存每个向量则会变得很麻烦。毕竟，我们可能有数百个参数散布在各处。因此，深度学习框架提供了内置函数来保存和加载整个网络。需要注意的一个重要细节是，这将<font color="Red">这将保存模型的参数而不是保存整个模型</font>。例如，如果我们有一个3层多层感知机，我们需要单独指定结构。因为模型本身可以包含任意代码，所以模型本身难以序列化。因此，为了恢复模型，我们需要用代码生成结构，然后从磁盘加载参数。让我们从熟悉的多层感知机开始尝试一下。

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden =nn.Linear(20,256)
        self.out =nn.Linear(256,10)
    
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

net =MLP()
X=torch.randn(size=(2,20))
Y =net(X)

接下来将模型的参数存储为一个叫做mlp.params的文件

In [8]:
net.state_dict()
# net.eval()

OrderedDict([('hidden.weight',
              tensor([[ 0.2175,  0.1988,  0.1089,  ...,  0.1514,  0.0749,  0.2087],
                      [ 0.1554, -0.2078,  0.0922,  ...,  0.0243,  0.1481,  0.0372],
                      [ 0.0973,  0.1889,  0.1612,  ...,  0.0292,  0.0430, -0.1021],
                      ...,
                      [ 0.2136,  0.2220, -0.0263,  ...,  0.0389, -0.0835,  0.0212],
                      [-0.0942,  0.1143, -0.1051,  ...,  0.1717,  0.1584, -0.0103],
                      [-0.1927, -0.0554, -0.0049,  ..., -0.1432,  0.0686,  0.1330]])),
             ('hidden.bias',
              tensor([-0.1409,  0.0276, -0.0751, -0.0830,  0.1467,  0.0830, -0.0091,  0.0989,
                       0.2019,  0.0361,  0.0194,  0.0333,  0.0781,  0.0566, -0.2188,  0.1257,
                      -0.0580,  0.1853, -0.0258, -0.2179,  0.1819,  0.1162, -0.0924,  0.1058,
                       0.0018, -0.0748, -0.1987,  0.0053, -0.1178,  0.0719,  0.1982,  0.0054,
                      -0.0864,

In [9]:
torch.save(net.state_dict(),'mlp.params')

In [10]:
clone=MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=10, bias=True)
)

In [11]:
Y_clone=clone(X)
Y_clone ==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])